<a href="https://colab.research.google.com/github/angeline2422/kaggle/blob/main/new_deep_feature_concatenation_with_two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle


In [2]:
from google.colab import files

# Upload the Kaggle API key (kaggle.json) that you downloaded
uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/


In [4]:
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle datasets download -d tourist55/alzheimers-dataset-4-class-of-images


 62% 21.0M/34.1M [00:00<00:00, 113MB/s] 
100% 34.1M/34.1M [00:00<00:00, 125MB/s]


In [6]:
!unzip /content/alzheimers-dataset-4-class-of-images.zip

Streaming output truncated to the last 5000 lines.
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem207.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem208.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem209.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem21.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem210.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem211.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem212.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem213.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem214.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem215.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem216.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem217.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem218.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/m

In [7]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.3/612.3 kB 6.0 MB/s eta 0:00:00


In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from distutils.dir_util import copy_tree, remove_tree
from PIL import Image
from random import randint
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow_addons as tfa
from keras.utils import plot_model
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D
print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.14.0


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


**DATA AUGMENTATION**

In [9]:
#base_dir = "/content/dataset"
base_dir = "/content/Alzheimer_s Dataset/"
root_dir = "./"
test_dir = base_dir + "test/"
train_dir = base_dir + "train/"
work_dir = root_dir + "dataset/"

if os.path.exists(work_dir):
    remove_tree(work_dir)


os.mkdir(work_dir)
copy_tree(train_dir, work_dir)
copy_tree(test_dir, work_dir)
print("Working Directory Contents:", os.listdir(work_dir))

Working Directory Contents: ['ModerateDemented', 'VeryMildDemented', 'NonDemented', 'MildDemented']


In [10]:


CLASSES = [ 'NonDemented',
            'VeryMildDemented',
            'MildDemented',
            'ModerateDemented']

IMG_SIZE = 150
IMAGE_SIZE = [150, 150]
DIM = (IMG_SIZE, IMG_SIZE)

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
ZOOM = [.99, 1.01]
BRIGHT_RANGE = [0.8, 1.2]
HORZ_FLIP = True
FILL_MODE = "constant"
DATA_FORMAT = "channels_last"
# Define the ImageDataGenerator with augmentation parameters
image_data_generator = ImageDataGenerator(
rescale=1.0 / 255,
brightness_range=BRIGHT_RANGE,
zoom_range=ZOOM,
data_format=DATA_FORMAT,
fill_mode=FILL_MODE,
horizontal_flip=HORZ_FLIP
)
# Create a generator for the training data from the working directory
train_data_gen = image_data_generator.flow_from_directory(
directory=work_dir,
target_size=IMAGE_SIZE,
batch_size=6500
                                                           ,
shuffle=False
)


Found 6400 images belonging to 4 classes.


In [12]:
#Retrieving the data from the ImageDataGenerator iterator

train_data, train_labels = train_data_gen.next()

In [13]:
#Getting to know the dimensions of our dataset

print(train_data.shape, train_labels.shape)

(6400, 150, 150, 3) (6400, 4)


In [14]:
#Performing over-sampling of the data, since the classes are imbalanced

sm = SMOTE(random_state=42)

train_data, train_labels = sm.fit_resample(train_data.reshape(-1, IMG_SIZE * IMG_SIZE * 3), train_labels)

train_data = train_data.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

print(train_data.shape, train_labels.shape)


(12800, 150, 150, 3) (12800, 4)


In [ ]:
!pip install tensorflow.keras.applications.VGG

ERROR: Could not find a version that satisfies the requirement tensorflow.keras.applications.VGG (from versions: none)
ERROR: No matching distribution found for tensorflow.keras.applications.VGG


In [15]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.applications import DenseNet121, VGG16

def custom_densenet(input_tensor, num_layers=5):
    base_model = DenseNet121(include_top=False, weights='imagenet', input_tensor=input_tensor)
    x = base_model.layers[-1].output
    for _ in range(num_layers):
        x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    return Model(inputs=input_tensor, outputs=x)

def custom_vgg16(input_tensor, num_layers=5):
    base_model = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)
    x = base_model.layers[-1].output
    for _ in range(num_layers):
        x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    return Model(inputs=input_tensor, outputs=x)

def deep_feature_concatenation():
    input_layer = Input(shape=(150, 150, 3))

    custom_densenet_model = custom_densenet(input_layer, num_layers=5)
    custom_vgg16_model = custom_vgg16(input_layer, num_layers=5)

    features_densenet = GlobalAveragePooling2D()(custom_densenet_model.output)
    features_vgg16 = GlobalAveragePooling2D()(custom_vgg16_model.output)

    features = concatenate([features_densenet, features_vgg16])

    x = Dense(512, activation='relu')(features)
    x = Dense(256, activation='relu')(x)
    x = Dense(4, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=x)

    return model

model = deep_feature_concatenation()
model.summary()


58889256/58889256 [==============================] - 0s 0us/step
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 156, 156, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 75, 75, 64)           9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati

In [ ]:
!pip install plotting


In [ ]:
pip install --upgrade pandas


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pandas

In [ ]:
pip install --upgrade pandas pyarrow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 34.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 9.0.0
    Uninstalling pyarrow-9.0.0:
      Successfully uninstalled pyarrow-9.0.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
google-colab 1.0.0 requires pandas==1.5.3, but 

In [ ]:
pip install --upgrade tensorflow keras


In [ ]:
from tensorflow.keras.utils import plot_model


In [16]:
































import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

import os
from distutils.dir_util import copy_tree, remove_tree

from PIL import Image
from random import randint

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow_addons as tfa
from keras.utils import plot_model

from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D

In [17]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc')
]

In [18]:
#Splitting the data into train, test, and validation sets
#Veriyi train, test ve validation olarak ayıralım
train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = model.fit(train_data, train_labels, epochs=20, batch_size=8, validation_data=(val_data, val_labels))


# Make predictions on the testing data
#y_pred = model.predict(X_test)

Epoch 1/20


In [ ]:
#Evaluating the model on the data

test_scores = model.evaluate(test_data, test_labels)
print("Testing Accuracy: %.2f%%"%(test_scores[1] * 100))

In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support

def showconfusionmatrix(cm, model_name):
    label_map= [
        'NonDemented',
        'VeryMildDemented',
        'MildDemented',
        'ModerateDemented'
    ]
    pl.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, cmap="rocket", xticklabels=label_map, yticklabels=label_map)
    pl.title('CM- ' + model_name)

def showmetrics(true, pred, model_name):
    accuracy = accuracy_score(true, pred)
    p, r, f, s = precision_recall_fscore_support(true, test_scores, average = 'weighted')
    cm = confusion_matrix(true, pred)
    TP = np.diag(cm)
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = [np.sum(cm), np.sum(cm), np.sum(cm), np.sum(cm)]
    TN = TN - (TP + FP + FN)
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    TPR = np.average(TPR, axis=0)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    TNR = np.average(TNR, axis=0)

    print (cm)
    print ("    ----Model---- ", model_name)
    print ("Accuracy         = ", accuracy)
    print ("Precision        = ", p)
    print ("Recall           = ", r)
    print ("F1_score         = ", f)
    print ("TP               = ", TP)
    print ("FP               = ", FP)
    print ("FN               = ", FN)
    print ("TN               = ", TN)
    print ("Mean Sensitivity = ", TPR)
    print ("Mean Specificity = ", TNR)
    cm = confusion_matrix(true, pred)
    showconfusionmatrix(cm, model_name)
    cm = confusion_matrix(true, pred, normalize='true')
    showconfusionmatrix(cm, model_name)

In [ ]:
print (test_labels)

In [ ]:
showmetrics(test_labels, pred_labels, '3DCNN')
